In [1]:
import numpy as np
import pandas as pd
import re
import ast

# Merge the two dataframes

In [26]:
anime_df = pd.read_csv('data/anime_df.csv')
anime_df.head()

,id,title,picture_url
0,51535,Shingeki no Kyojin: The Final Season - Kankets...,https://api-cdn.myanimelist.net/images/anime/1...
1,5114,Fullmetal Alchemist: Brotherhood,https://api-cdn.myanimelist.net/images/anime/1...
2,41467,Bleach: Sennen Kessen-hen,https://api-cdn.myanimelist.net/images/anime/1...
3,9253,Steins;Gate,https://api-cdn.myanimelist.net/images/anime/1...
4,28977,Gintama°,https://api-cdn.myanimelist.net/images/anime/3...


In [3]:
anime_details_df = pd.read_csv('data/anime_details_df.csv')
anime_details_df.head()

,id,alternative_titles,synopsis,mean,rank,popularity,nsfw,media_type,status,genres,num_episodes,start_season,source,average_episode_duration,rating,background,studios,statistics
0,51535,{'synonyms': ['Shingeki no Kyojin: The Final S...,The conclusion to Shingeki no Kyojin.,9.11,1,540,white,special,currently_airing,"[{'id': 1, 'name': 'Action'}, {'id': 8, 'name'...",2,"{'year': 2023, 'season': 'winter'}",manga,3690,r,NaN,"[{'id': 569, 'name': 'MAPPA'}]","{'status': {'watching': '188176', 'completed':..."
1,5114,{'synonyms': ['Hagane no Renkinjutsushi: Fullm...,After a horrific alchemy experiment goes wrong...,9.11,2,3,white,tv,finished_airing,"[{'id': 1, 'name': 'Action'}, {'id': 2, 'name'...",64,"{'year': 2009, 'season': 'spring'}",manga,1460,r,NaN,"[{'id': 4, 'name': 'Bones'}]","{'status': {'watching': '241806', 'completed':..."
2,41467,{'synonyms': ['Bleach: Thousand-Year Blood War...,Substitute Soul Reaper Ichigo Kurosaki spends ...,9.08,3,495,white,tv,finished_airing,"[{'id': 1, 'name': 'Action'}, {'id': 2, 'name'...",13,"{'year': 2022, 'season': 'fall'}",manga,1471,r,Bleach: Sennen Kessen-hen adapts volume 55 to ...,"[{'id': 1, 'name': 'Pierrot'}]","{'status': {'watching': '95738', 'completed': ..."
3,9253,"{'synonyms': [], 'en': 'Steins;Gate', 'ja': 'S...",Eccentric scientist Rintarou Okabe has a never...,9.08,4,13,white,tv,finished_airing,"[{'id': 8, 'name': 'Drama'}, {'id': 40, 'name'...",24,"{'year': 2011, 'season': 'spring'}",visual_novel,1460,pg_13,Steins;Gate is based on 5pb. and Nitroplus' vi...,"[{'id': 314, 'name': 'White Fox'}]","{'status': {'watching': '157936', 'completed':..."
4,28977,"{'synonyms': [""Gintama' (2015)""], 'en': 'Ginta...","Gintoki, Shinpachi, and Kagura return as the f...",9.07,5,334,white,tv,finished_airing,"[{'id': 1, 'name': 'Action'}, {'id': 4, 'name'...",51,"{'year': 2015, 'season': 'spring'}",manga,1440,pg_13,NaN,"[{'id': 1258, 'name': 'Bandai Namco Pictures'}]","{'status': {'watching': '64460', 'completed': ..."


In [4]:
df = anime_df.merge(anime_details_df, on='id', how='inner')
df.head()

,id,title,picture_url,alternative_titles,synopsis,mean,rank,popularity,nsfw,media_type,status,genres,num_episodes,start_season,source,average_episode_duration,rating,background,studios,statistics
0,51535,Shingeki no Kyojin: The Final Season - Kankets...,https://api-cdn.myanimelist.net/images/anime/1...,{'synonyms': ['Shingeki no Kyojin: The Final S...,The conclusion to Shingeki no Kyojin.,9.11,1,540,white,special,currently_airing,"[{'id': 1, 'name': 'Action'}, {'id': 8, 'name'...",2,"{'year': 2023, 'season': 'winter'}",manga,3690,r,NaN,"[{'id': 569, 'name': 'MAPPA'}]","{'status': {'watching': '188176', 'completed':..."
1,5114,Fullmetal Alchemist: Brotherhood,https://api-cdn.myanimelist.net/images/anime/1...,{'synonyms': ['Hagane no Renkinjutsushi: Fullm...,After a horrific alchemy experiment goes wrong...,9.11,2,3,white,tv,finished_airing,"[{'id': 1, 'name': 'Action'}, {'id': 2, 'name'...",64,"{'year': 2009, 'season': 'spring'}",manga,1460,r,NaN,"[{'id': 4, 'name': 'Bones'}]","{'status': {'watching': '241806', 'completed':..."
2,41467,Bleach: Sennen Kessen-hen,https://api-cdn.myanimelist.net/images/anime/1...,{'synonyms': ['Bleach: Thousand-Year Blood War...,Substitute Soul Reaper Ichigo Kurosaki spends ...,9.08,3,495,white,tv,finished_airing,"[{'id': 1, 'name': 'Action'}, {'id': 2, 'name'...",13,"{'year': 2022, 'season': 'fall'}",manga,1471,r,Bleach: Sennen Kessen-hen adapts volume 55 to ...,"[{'id': 1, 'name': 'Pierrot'}]","{'status': {'watching': '95738', 'completed': ..."
3,9253,Steins;Gate,https://api-cdn.myanimelist.net/images/anime/1...,"{'synonyms': [], 'en': 'Steins;Gate', 'ja': 'S...",Eccentric scientist Rintarou Okabe has a never...,9.08,4,13,white,tv,finished_airing,"[{'id': 8, 'name': 'Drama'}, {'id': 40, 'name'...",24,"{'year': 2011, 'season': 'spring'}",visual_novel,1460,pg_13,Steins;Gate is based on 5pb. and Nitroplus' vi...,"[{'id': 314, 'name': 'White Fox'}]","{'status': {'watching': '157936', 'completed':..."
4,28977,Gintama°,https://api-cdn.myanimelist.net/images/anime/3...,"{'synonyms': [""Gintama' (2015)""], 'en': 'Ginta...","Gintoki, Shinpachi, and Kagura return as the f...",9.07,5,334,white,tv,finished_airing,"[{'id': 1, 'name': 'Action'}, {'id': 4, 'name'...",51,"{'year': 2015, 'season': 'spring'}",manga,1440,pg_13,NaN,"[{'id': 1258, 'name': 'Bandai Namco Pictures'}]","{'status': {'watching': '64460', 'completed': ..."


# EDA

In [5]:
df.shape

(10000, 20)

In [6]:
# check duplicated rows
df['id'].duplicated().value_counts()

False    10000
Name: id, dtype: int64

In [7]:
# check missing values
df.isnull().sum()

id                             0
title                          0
picture_url                    0
alternative_titles             0
synopsis                     180
mean                           0
rank                           0
popularity                     0
nsfw                           0
media_type                     0
status                         0
genres                        17
num_episodes                   0
start_season                 139
source                       975
average_episode_duration       0
rating                        58
background                  8304
studios                        0
statistics                     0
dtype: int64

# Data Cleaning

In [8]:
# drop rows where the synopsis/plot is null because it's an important feature for content-based-filtering
df.dropna(subset=['synopsis'], inplace=True)
# drop the background column since a lot of data are missing
df.drop('background', axis=1, inplace=True)

In [9]:
df.shape

(9820, 19)

In [10]:
# normalize the alternative_titles column
df['alternative_titles'] = df['alternative_titles'].apply(lambda x: ast.literal_eval(x))
df = df.join(pd.json_normalize(df.pop('alternative_titles')))

df.head(3)

,id,title,picture_url,synopsis,mean,rank,popularity,nsfw,media_type,status,...,num_episodes,start_season,source,average_episode_duration,rating,studios,statistics,synonyms,en,ja
0,51535,Shingeki no Kyojin: The Final Season - Kankets...,https://api-cdn.myanimelist.net/images/anime/1...,The conclusion to Shingeki no Kyojin.,9.11,1,540,white,special,currently_airing,...,2,"{'year': 2023, 'season': 'winter'}",manga,3690,r,"[{'id': 569, 'name': 'MAPPA'}]","{'status': {'watching': '188176', 'completed':...","[Shingeki no Kyojin: The Final Season Part 3, ...",Attack on Titan: Final Season - The Final Chap...,進撃の巨人 The Final Season完結編
1,5114,Fullmetal Alchemist: Brotherhood,https://api-cdn.myanimelist.net/images/anime/1...,After a horrific alchemy experiment goes wrong...,9.11,2,3,white,tv,finished_airing,...,64,"{'year': 2009, 'season': 'spring'}",manga,1460,r,"[{'id': 4, 'name': 'Bones'}]","{'status': {'watching': '241806', 'completed':...",[Hagane no Renkinjutsushi: Fullmetal Alchemist...,Fullmetal Alchemist: Brotherhood,鋼の錬金術師 FULLMETAL ALCHEMIST
2,41467,Bleach: Sennen Kessen-hen,https://api-cdn.myanimelist.net/images/anime/1...,Substitute Soul Reaper Ichigo Kurosaki spends ...,9.08,3,495,white,tv,finished_airing,...,13,"{'year': 2022, 'season': 'fall'}",manga,1471,r,"[{'id': 1, 'name': 'Pierrot'}]","{'status': {'watching': '95738', 'completed': ...",[Bleach: Thousand-Year Blood War Arc],Bleach: Thousand-Year Blood War,BLEACH 千年血戦篇


In [11]:
# get rid of unnecessary characters in the genres column
df['genres'] = df['genres'].apply(lambda x: re.sub('[\[\]{}\'0-9:,]', '', str(x)))
df['genres'] = df['genres'].apply(lambda x: str(x).replace('id', '').replace('name', ''))

df.head(3)

,id,title,picture_url,synopsis,mean,rank,popularity,nsfw,media_type,status,...,num_episodes,start_season,source,average_episode_duration,rating,studios,statistics,synonyms,en,ja
0,51535,Shingeki no Kyojin: The Final Season - Kankets...,https://api-cdn.myanimelist.net/images/anime/1...,The conclusion to Shingeki no Kyojin.,9.11,1,540,white,special,currently_airing,...,2,"{'year': 2023, 'season': 'winter'}",manga,3690,r,"[{'id': 569, 'name': 'MAPPA'}]","{'status': {'watching': '188176', 'completed':...","[Shingeki no Kyojin: The Final Season Part 3, ...",Attack on Titan: Final Season - The Final Chap...,進撃の巨人 The Final Season完結編
1,5114,Fullmetal Alchemist: Brotherhood,https://api-cdn.myanimelist.net/images/anime/1...,After a horrific alchemy experiment goes wrong...,9.11,2,3,white,tv,finished_airing,...,64,"{'year': 2009, 'season': 'spring'}",manga,1460,r,"[{'id': 4, 'name': 'Bones'}]","{'status': {'watching': '241806', 'completed':...",[Hagane no Renkinjutsushi: Fullmetal Alchemist...,Fullmetal Alchemist: Brotherhood,鋼の錬金術師 FULLMETAL ALCHEMIST
2,41467,Bleach: Sennen Kessen-hen,https://api-cdn.myanimelist.net/images/anime/1...,Substitute Soul Reaper Ichigo Kurosaki spends ...,9.08,3,495,white,tv,finished_airing,...,13,"{'year': 2022, 'season': 'fall'}",manga,1471,r,"[{'id': 1, 'name': 'Pierrot'}]","{'status': {'watching': '95738', 'completed': ...",[Bleach: Thousand-Year Blood War Arc],Bleach: Thousand-Year Blood War,BLEACH 千年血戦篇


In [12]:
# normalize the start_season column
df['start_season'] = df['start_season'].replace(np.nan, 'None')
df['start_season'] = df['start_season'].apply(lambda x: ast.literal_eval(x))
df = df.join(pd.json_normalize(df.pop('start_season')))

df.head(3)

,id,title,picture_url,synopsis,mean,rank,popularity,nsfw,media_type,status,...,source,average_episode_duration,rating,studios,statistics,synonyms,en,ja,year,season
0,51535,Shingeki no Kyojin: The Final Season - Kankets...,https://api-cdn.myanimelist.net/images/anime/1...,The conclusion to Shingeki no Kyojin.,9.11,1,540,white,special,currently_airing,...,manga,3690,r,"[{'id': 569, 'name': 'MAPPA'}]","{'status': {'watching': '188176', 'completed':...","[Shingeki no Kyojin: The Final Season Part 3, ...",Attack on Titan: Final Season - The Final Chap...,進撃の巨人 The Final Season完結編,2023.0,winter
1,5114,Fullmetal Alchemist: Brotherhood,https://api-cdn.myanimelist.net/images/anime/1...,After a horrific alchemy experiment goes wrong...,9.11,2,3,white,tv,finished_airing,...,manga,1460,r,"[{'id': 4, 'name': 'Bones'}]","{'status': {'watching': '241806', 'completed':...",[Hagane no Renkinjutsushi: Fullmetal Alchemist...,Fullmetal Alchemist: Brotherhood,鋼の錬金術師 FULLMETAL ALCHEMIST,2009.0,spring
2,41467,Bleach: Sennen Kessen-hen,https://api-cdn.myanimelist.net/images/anime/1...,Substitute Soul Reaper Ichigo Kurosaki spends ...,9.08,3,495,white,tv,finished_airing,...,manga,1471,r,"[{'id': 1, 'name': 'Pierrot'}]","{'status': {'watching': '95738', 'completed': ...",[Bleach: Thousand-Year Blood War Arc],Bleach: Thousand-Year Blood War,BLEACH 千年血戦篇,2022.0,fall


In [13]:
# get rid of unnecessary characters in the studios column
df['studios'] = df['studios'].apply(lambda x: re.sub('[\[\]{}\'0-9:,]', '', str(x)))
df['studios'] = df['studios'].apply(lambda x: str(x).replace('id', '').replace('name', ''))

df.head(3)

,id,title,picture_url,synopsis,mean,rank,popularity,nsfw,media_type,status,...,source,average_episode_duration,rating,studios,statistics,synonyms,en,ja,year,season
0,51535,Shingeki no Kyojin: The Final Season - Kankets...,https://api-cdn.myanimelist.net/images/anime/1...,The conclusion to Shingeki no Kyojin.,9.11,1,540,white,special,currently_airing,...,manga,3690,r,MAPPA,"{'status': {'watching': '188176', 'completed':...","[Shingeki no Kyojin: The Final Season Part 3, ...",Attack on Titan: Final Season - The Final Chap...,進撃の巨人 The Final Season完結編,2023.0,winter
1,5114,Fullmetal Alchemist: Brotherhood,https://api-cdn.myanimelist.net/images/anime/1...,After a horrific alchemy experiment goes wrong...,9.11,2,3,white,tv,finished_airing,...,manga,1460,r,Bones,"{'status': {'watching': '241806', 'completed':...",[Hagane no Renkinjutsushi: Fullmetal Alchemist...,Fullmetal Alchemist: Brotherhood,鋼の錬金術師 FULLMETAL ALCHEMIST,2009.0,spring
2,41467,Bleach: Sennen Kessen-hen,https://api-cdn.myanimelist.net/images/anime/1...,Substitute Soul Reaper Ichigo Kurosaki spends ...,9.08,3,495,white,tv,finished_airing,...,manga,1471,r,Pierrot,"{'status': {'watching': '95738', 'completed': ...",[Bleach: Thousand-Year Blood War Arc],Bleach: Thousand-Year Blood War,BLEACH 千年血戦篇,2022.0,fall


In [14]:
# normalize the statistics column
df['statistics'] = df['statistics'].apply(lambda x: ast.literal_eval(x))
df = df.join(pd.json_normalize(df.pop('statistics')))

df.head(3)

,id,title,picture_url,synopsis,mean,rank,popularity,nsfw,media_type,status,...,en,ja,year,season,num_list_users,status.watching,status.completed,status.on_hold,status.dropped,status.plan_to_watch
0,51535,Shingeki no Kyojin: The Final Season - Kankets...,https://api-cdn.myanimelist.net/images/anime/1...,The conclusion to Shingeki no Kyojin.,9.11,1,540,white,special,currently_airing,...,Attack on Titan: Final Season - The Final Chap...,進撃の巨人 The Final Season完結編,2023.0,winter,382102.0,188176,724,18741,1009,173452
1,5114,Fullmetal Alchemist: Brotherhood,https://api-cdn.myanimelist.net/images/anime/1...,After a horrific alchemy experiment goes wrong...,9.11,2,3,white,tv,finished_airing,...,Fullmetal Alchemist: Brotherhood,鋼の錬金術師 FULLMETAL ALCHEMIST,2009.0,spring,3126275.0,241806,2260153,106496,52671,465149
2,41467,Bleach: Sennen Kessen-hen,https://api-cdn.myanimelist.net/images/anime/1...,Substitute Soul Reaper Ichigo Kurosaki spends ...,9.08,3,495,white,tv,finished_airing,...,Bleach: Thousand-Year Blood War,BLEACH 千年血戦篇,2022.0,fall,415659.0,95738,176566,4876,3407,135072


In [15]:
df.columns

Index(['id', 'title', 'picture_url', 'synopsis', 'mean', 'rank', 'popularity',
       'nsfw', 'media_type', 'status', 'genres', 'num_episodes', 'source',
       'average_episode_duration', 'rating', 'studios', 'synonyms', 'en', 'ja',
       'year', 'season', 'num_list_users', 'status.watching',
       'status.completed', 'status.on_hold', 'status.dropped',
       'status.plan_to_watch'],
      dtype='object')

In [16]:
# rename columns
df = df.rename(columns={'synonyms':'alternative_titles', 'en':'english_title', 'ja':'japanese_title',
                        'status.watching':'num_of_ppl_watching','status.completed':'num_of_ppl_completed', 
                        'status.on_hold':'num_of_ppl_on_hold', 'status.dropped':'num_of_ppl_dropped',
                        'status.plan_to_watch':'num_of_ppl_plan_to_watch'})

# Text preprocessing